<a href="https://colab.research.google.com/github/TuanKiet1774/ThuatToanAI/blob/main/DetectPest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Kết nối với Drive

Kết nối với Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Giải nén file Zip đến thư mục chỉ định

In [ ]:
%%bash
unzip /content/drive/MyDrive/OBD-HETTNT/Pest.v2i.yolov11.zip -d /content/drive/MyDrive/OBD-HETTNT

##Visual Anotate

Công thức tính:
(class) (tâm chéo (x,y)) (chiều dài) (chiều rộng)
Chia cho tỷ lệ khung hình

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

# Path to the image and its corresponding annotation file
image_path = "/content/drive/MyDrive/OBD-HETTNT/train/images/2-Figure1-1_png.rf.59941ba058b3226147c1a79f549dbe93.jpg"
annotation_path = "/content/drive/MyDrive/OBD-HETTNT/train/labels/2-Figure1-1_png.rf.59941ba058b3226147c1a79f549dbe93.txt"

# Load the image
image = cv2.imread(image_path)
height, width, _ = image.shape

# Define colors for different class IDs (just using 3 classes here for example)
colors = {
    0: (0, 255, 0),  # Green for class 0
    1: (0, 0, 255),  # Red for class 1
    2: (255, 0, 0)   # Blue for class 2
}

# List of class names (partial Coco names for illustration, you can use the full list)
coco_names = ['African Armyworm', 'bean beetles', 'cowpea aphids', 'fall armyworm', 'maize weevil', 'white flies']

# Read the annotation file
with open(annotation_path, "r") as file:
    for line in file.readlines():
        # Each line: class_id x_center y_center width height
        parts = line.strip().split()
        class_id = int(parts[0])  # Class ID
        x_center = float(parts[1]) * width
        y_center = float(parts[2]) * height
        box_width = float(parts[3]) * width
        box_height = float(parts[4]) * height

        # Convert YOLO format to bounding box coordinates
        xmin = int(x_center - box_width / 2)
        ymin = int(y_center - box_height / 2)
        xmax = int(x_center + box_width / 2)
        ymax = int(y_center + box_height / 2)

        # Draw the rectangle and label on the image
        color = colors.get(class_id, (255, 255, 255))  # Default color: white
        class_name = coco_names[class_id]  # Get class name from coco_names
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 2)
        cv2.putText(image, class_name, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Convert BGR (OpenCV format) to RGB (Matplotlib format)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display and save the image
plt.imshow(image_rgb)
plt.axis('off')  # Remove axes for better visual clarity
#plt.savefig('visual_IP038000453_with_labels.jpg', bbox_inches='tight', pad_inches=0)  # Save before showing
plt.show()
plt.close()

##Tải Yolo11 Ultralytics

Clone repo Ultrlytics về Drive

In [ ]:
%%bash
cd /content/drive/MyDrive/OBD-HETTNT
git clone https://github.com/ultralytics/ultralytics.git

Tải thư viện ultralytics

In [ ]:
%%bash
pip install ultralytics

##Training tập dữ liệu

In [ ]:
%cd /content/drive/MyDrive/OBD-HETTNT/ultralytics
from ultralytics import YOLO
model = YOLO("yolo11n.yaml")
model.train(data="/content/drive/MyDrive/OBD-HETTNT/data.yaml", epochs=50, imgsz=640)

##Train với hệ số có sẵn

In [ ]:
%cd /content/drive/MyDrive/OBD-HETTNT/ultralytics
from ultralytics import YOLO
model = YOLO("yolo11n.yaml")
model = YOLO("yolo11n.pt")
model = YOLO("yolo11n.yaml").load("yolo11n.pt")
model.train(data="/content/drive/MyDrive/OBD-HETTNT/data.yaml", epochs=50, imgsz=640)

Predict

In [ ]:
from ultralytics import YOLO
import numpy as np
import cv2
import os
import glob

# Load model
model = YOLO("/content/drive/MyDrive/OBD-HETTNT/ultralytics/runs/detect/train2/weights/best.pt")

# Get class names from the model
names = model.names

# Generate palette
def get_palette(num_classes):
    np.random.seed(42)
    return [tuple(np.random.randint(0, 256, size=3).tolist()) for _ in range(num_classes)]

palette = get_palette(len(names))

# Input/output folders
input_dir = '/content/drive/MyDrive/OBD-HETTNT/test/images'
output_dir = '/content/drive/MyDrive/OBD-HETTNT/Predict_DetectPest'
os.makedirs(output_dir, exist_ok=True)

# Get all image paths
image_paths = glob.glob(os.path.join(input_dir, '*.jpg'))

for img_path in image_paths:
    image = cv2.imread(img_path)
    img_name = os.path.basename(img_path)
    out_path = os.path.join(output_dir, img_name)

    results = model(img_path)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])
        cls_name = names[cls_id]  # Get class name using the class ID

        color = palette[cls_id % len(palette)]

        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        text = f'{cls_name} {conf:.2f}' # Use cls_name here
        cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)


    cv2.imwrite(out_path, image)

print(f'✅ Done. Visualized results saved to: {output_dir}')